In [ ]:

import streamlit as st
import google.generativeai as genai
import PyPDF2
import io
import json
from pathlib import Path
from textblob import TextBlob
from fpdf import FPDF
import datetime
import random

# --- Streamlit UI Setup ---
st.set_page_config(page_title="Chatbot at Your Service 🤖", page_icon="🤖")
st.title("🤖 Chatbot At Your Service 🤖")

# --- JSON Files for Responses ---
GREETINGS_FILE = "greetings_responses.json"
OPENING_QUESTIONS_FILE = "opening_questions.json"
FEEDBACK_FILE = "feedback_responses.json"
FEEDBACK_MEMORY_FILE = "feedback/feedback_memory.json"

# Initialize JSON files if they don't exist
def initialize_json_files():
    default_greetings = {
        "responses": [
            "I'm doing great, thanks for asking! How about you?",
            "Wonderful to hear from you! I'm doing well today.",
            "I'm feeling fantastic! Hope you're having a good day too.",
            "All good here! How's everything with you?",
            "I'm excellent, thank you! And how are you doing?"
        ],
        "followups": [
            "What can I help you with today?",
            "How may I assist you right now?",
            "What would you like to explore today?",
            "How can I make your day easier?",
            "What information would be helpful for you?"
        ]
    }

    default_opening_questions = {
        "document_questions": [
            "What would you like to know from this document?",
            "Which part of the document should we look at together?",
            "How can I help you understand this material better?",
            "What information would be most useful to you from this file?",
            "Which section would you like me to explain?"
        ],
        "general_questions": [
            "How can I support you today?",
            "What would you like to discuss?",
            "How may I be helpful to you right now?",
            "What's on your mind that I can help with?",
            "What information are you looking for?"
        ]
    }

    default_feedback = {
        "end_phrases": [
            "no, thank you",
            "i am good",
            "thats all",
            "thank you so much for your assistance",
            "no, thanks",
            "that's all for now",
            "i'm done",
            "nothing else"
        ],
        "goodbye": [
            "Goodbye! Wishing you a wonderful day ahead!",
            "Farewell! May your day be filled with joy!",
            "Until next time! Take care and stay awesome!",
            "Thank you for chatting! Hope to talk again soon!",
            "Goodbye! Remember you're doing great!"
        ],
        "rating_responses": {
            "low": [
                "I truly appreciate your honest feedback - I'll work hard to improve.",
                "Thank you for letting me know how I can do better next time.",
                "Your feedback is valuable to me. I'll learn from this experience."
            ],
            "medium": [
                "Thanks for your rating! I'll keep working to improve.",
                "I appreciate your feedback - it helps me grow!",
                "Thank you! I'll use this to make our next chat even better."
            ],
            "high": [
                "You've made my day with this wonderful rating! Thank you!",
                "I'm so grateful for your kind rating! You're amazing!",
                "Thank you for the fantastic rating! It means the world to me!"
            ]
        }
    }

    Path("feedback").mkdir(parents=True, exist_ok=True)

    if not Path(GREETINGS_FILE).exists():
        with open(GREETINGS_FILE, 'w') as f:
            json.dump(default_greetings, f, indent=2)

    if not Path(OPENING_QUESTIONS_FILE).exists():
        with open(OPENING_QUESTIONS_FILE, 'w') as f:
            json.dump(default_opening_questions, f, indent=2)

    if not Path(FEEDBACK_FILE).exists():
        with open(FEEDBACK_FILE, 'w') as f:
            json.dump(default_feedback, f, indent=2)

initialize_json_files()

# --- Sidebar Configuration ---
with st.sidebar:
    st.header("⚙️ Settings")
    api_key = st.text_input("🔑 Enter Gemini API Key:", type="password")
    if api_key:
        st.success("API Key Set! ✅")
    st.markdown("[Get Gemini API Key](https://aistudio.google.com/app/apikey)")

    uploaded_file = st.file_uploader("📄 Upload a PDF", type=["pdf"])

    if st.button("🧹 Clear Memory"):
        if Path(FEEDBACK_MEMORY_FILE).exists():
            Path(FEEDBACK_MEMORY_FILE).unlink()
        st.session_state.clear()
        st.rerun()

    if st.button("💾 Save Chat"):
        if "messages" in st.session_state and len(st.session_state.messages) > 0:
            save_chat_to_file()
        else:
            st.warning("No conversation to save!")

# --- File Saving Functions ---
def save_chat_to_file():
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")

    txt_content = "\n".join([f"{m['role']}: {m['content']}" for m in st.session_state.messages])
    st.download_button(
        label="⬇️ Download as TXT",
        data=txt_content,
        file_name=f"chat_history_{timestamp}.txt",
        mime="text/plain"
    )

    pdf = FPDF()
    pdf.add_page()
    pdf.set_font("Arial", size=12)

    for message in st.session_state.messages:
        role = "You" if message["role"] == "user" else "Assistant"
        pdf.multi_cell(0, 10, f"{role}: {message['content']}")
        pdf.ln(5)

    pdf.output(f"chat_history_{timestamp}.pdf")
    with open(f"chat_history_{timestamp}.pdf", "rb") as f:
        st.download_button(
            label="⬇️ Download as PDF",
            data=f,
            file_name=f"chat_history_{timestamp}.pdf",
            mime="application/pdf"
        )

# --- Enhanced Emotional Intelligence ---
def analyze_sentiment(text):
    analysis = TextBlob(text)
    polarity = analysis.sentiment.polarity

    if polarity < -0.5:
        return "very_negative"
    elif -0.5 <= polarity < -0.2:
        return "negative"
    elif -0.2 <= polarity < 0.2:
        return "neutral"
    elif 0.2 <= polarity < 0.5:
        return "positive"
    else:
        return "very_positive"

def generate_emotional_response(user_input, context):
    sentiment = analyze_sentiment(user_input)

    emotional_responses = {
        "very_negative": [
            "I can hear how upsetting this is for you. Let me try to help...",
            "This sounds really painful. I want to support you through this...",
            "I can feel your frustration. Let's work on this together..."
        ],
        "negative": [
            "I understand this is difficult. Let me share what might help...",
            "This seems to be weighing on you. Maybe this information will help...",
            "I can tell this matters to you. Here's what I can share..."
        ],
        "neutral": [
            "Let me help with that information for you...",
            "I'd be happy to assist with that...",
            "Here's what I can tell you about that..."
        ],
        "positive": [
            "That's wonderful to hear! Let me tell you more about this...",
            "I'm glad you asked! Here's what you should know...",
            "Great question! Here's the information you're looking for..."
        ],
        "very_positive": [
            "Your excitement is contagious! Let me share this with you...",
            "I love your enthusiasm! Here's what you need to know...",
            "What a great question! Here's the information you wanted..."
        ]
    }

    return random.choice(emotional_responses.get(sentiment, [context])) + f" {context}"

# --- Enhanced Feedback System ---
def load_feedback_memory():
    try:
        if Path(FEEDBACK_MEMORY_FILE).exists():
            with open(FEEDBACK_MEMORY_FILE, "r") as f:
                return json.load(f)
        return {
            "corrections": {},
            "feedback_history": [],
            "low_rating_responses": {},
            "high_rating_responses": {}
        }
    except Exception as e:
        st.error(f"❌ Error loading feedback memory: {str(e)}")
        return {
            "corrections": {},
            "feedback_history": [],
            "low_rating_responses": {},
            "high_rating_responses": {}
        }

def save_feedback_memory(memory):
    try:
        Path(FEEDBACK_MEMORY_FILE).parent.mkdir(parents=True, exist_ok=True)
        with open(FEEDBACK_MEMORY_FILE, "w") as f:
            json.dump(memory, f, indent=2)
        return True
    except Exception as e:
        st.error(f"❌ Failed to save feedback memory: {str(e)}")
        return False

def get_improved_response(prompt, pdf_text=None):
    feedback_memory = load_feedback_memory()

    for stored_prompt, response in feedback_memory.get("low_rating_responses", {}).items():
        if prompt.lower() in stored_prompt.lower() or stored_prompt.lower() in prompt.lower():
            return response

    for stored_prompt, response in feedback_memory.get("high_rating_responses", {}).items():
        if prompt.lower() in stored_prompt.lower() or stored_prompt.lower() in prompt.lower():
            return response

    return None

def analyze_and_improve(feedback_rating):
    if not isinstance(feedback_rating, int) or feedback_rating < 1 or feedback_rating > 10:
        st.error("Invalid rating value")
        return False

    feedback_memory = load_feedback_memory()

    feedback_memory["feedback_history"].append({
        "rating": feedback_rating,
        "conversation": st.session_state.messages.copy(),
        "timestamp": datetime.datetime.now().isoformat()
    })

    if len(st.session_state.messages) >= 2:
        last_user_msg = next(
            (msg for msg in reversed(st.session_state.messages) if msg["role"] == "user"),
            None
        )
        last_assistant_msg = next(
            (msg for msg in reversed(st.session_state.messages) if msg["role"] == "assistant"),
            None
        )

        if last_user_msg and last_assistant_msg:
            if feedback_rating < 5:
                feedback_memory["low_rating_responses"][last_user_msg["content"]] = last_assistant_msg["content"]
            elif feedback_rating >= 5:
                feedback_memory["high_rating_responses"][last_user_msg["content"]] = last_assistant_msg["content"]

    return save_feedback_memory(feedback_memory)

# --- Chat Initialization ---
if "messages" not in st.session_state:
    st.session_state.messages = []
    st.session_state.awaiting_feedback = False
    st.session_state.awaiting_rating = False
    st.session_state.session_complete = False
    st.session_state.last_question = None
    st.session_state.last_response = None
    st.session_state.awaiting_save_confirmation = False

for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"], unsafe_allow_html=True)

# --- PDF Processing ---
def extract_text_from_pdf(pdf_file):
    text = ""
    try:
        pdf_reader = PyPDF2.PdfReader(io.BytesIO(pdf_file.read()))
        for page in pdf_reader.pages:
            text += page.extract_text()
        return text
    except Exception as e:
        st.error(f"❌ Error reading PDF: {str(e)}")
        return None

# --- Enhanced Response Generation ---
def generate_response(prompt, pdf_text=None):
    try:
        improved_response = get_improved_response(prompt, pdf_text)
        if improved_response:
            return improved_response

        genai.configure(api_key=api_key)
        model = genai.GenerativeModel('gemini-1.5-flash')

        context = f"PDF Content: {pdf_text}" if pdf_text else "No PDF context available"

        prompt_with_context = f"""
        User Question: {prompt}
        {context}

        Instructions:
        - First determine if the question is related to the document content
        - If the information is not in the document, say so clearly
        - Respond with perfect grammar and punctuation
        - Match the user's emotional tone
        - Be concise but thorough
        - Use natural, human-like language
        - If unsure, say the information isn't available
        """

        response = model.generate_content(prompt_with_context)
        generated_response = generate_emotional_response(prompt, response.text)

        no_info_phrases = [
            "not mentioned", "not available", "not found",
            "not provided", "no information", "not in the document",
            "not covered", "doesn't mention", "doesn't include",
            "not specified", "not discussed", "not addressed"
        ]

        if (any(phrase in response.text.lower() for phrase in no_info_phrases) or
            any(phrase in generated_response.lower() for phrase in no_info_phrases)):
            return "I am sorry but the information you are looking for is not available in the uploaded document. Please proceed asking information relevant to the uploaded file. Thank you!"

        return generated_response
    except Exception as e:
        return f"⚠️ Error: {str(e)}"

# --- Load JSON Responses ---
def load_json_responses(file_path):
    with open(file_path, 'r') as f:
        return json.load(f)

# --- Main Chat Logic ---
if uploaded_file:
    if "pdf_text" not in st.session_state:
        st.session_state.pdf_text = extract_text_from_pdf(uploaded_file)

    if st.session_state.pdf_text:
        if prompt := st.chat_input("💬 Ask me anything..."):
            greetings = load_json_responses(GREETINGS_FILE)
            opening_questions = load_json_responses(OPENING_QUESTIONS_FILE)
            feedback_responses = load_json_responses(FEEDBACK_FILE)

            if any(greet in prompt.lower() for greet in ["hello", "hi", "hey", "greetings", "how are you"]):
                response = random.choice(greetings["responses"])
                st.session_state.messages.append({"role": "user", "content": prompt})
                st.session_state.messages.append({"role": "assistant", "content": response})

                if uploaded_file:
                    followup = random.choice(opening_questions["document_questions"])
                else:
                    followup = random.choice(opening_questions["general_questions"])
                st.session_state.messages.append({"role": "assistant", "content": followup})
                st.rerun()

            elif prompt.lower().strip() in ["goodbye", "bye"]:
                st.session_state.messages.append({"role": "user", "content": prompt})
                goodbye_responses = [
                    "Bye bye!",
                    "It was a pleasure to be useful for your queries. Goodbye!"
                ]
                response = random.choice(goodbye_responses)
                st.session_state.messages.append({"role": "assistant", "content": response})

                save_prompt = "Do you wish to save our chat session? (Yes/No)"
                st.session_state.messages.append({"role": "assistant", "content": save_prompt})
                st.session_state.awaiting_save_confirmation = True
                st.rerun()

            elif st.session_state.get("awaiting_save_confirmation"):
                normalized_input = prompt.lower().strip()
                st.session_state.messages.append({"role": "user", "content": prompt})

                if normalized_input in ["yes", "y"]:
                    response = 'Please click on the "save chat" tab on the left side of the screen'
                    st.session_state.messages.append({"role": "assistant", "content": response})
                else:
                    response = "Once again thank you for using me for your queries. I wish you a beautiful and smiling day ahead!"
                    st.session_state.messages.append({"role": "assistant", "content": response})

                st.session_state.awaiting_save_confirmation = False
                st.session_state.session_complete = True
                st.rerun()

            elif st.session_state.awaiting_feedback:
                normalized_input = prompt.lower().strip()

                if normalized_input in ["no", "n"]:
                    st.session_state.messages.append({"role": "user", "content": prompt})
                    st.session_state.awaiting_feedback = False

                    rating_prompt = """
                    <div style='background-color:#f0f2f6; padding:10px; border-radius:5px;'>
                    <b>Please help me improve by rating our conversation (1-10):</b><br>
                    1 = Needs improvement, 10 = Excellent
                    </div>
                    """
                    st.session_state.messages.append({"role": "assistant", "content": rating_prompt})
                    st.session_state.awaiting_rating = True
                    st.rerun()

                elif normalized_input in ["yes", "y"]:
                    st.session_state.messages.append({"role": "user", "content": prompt})
                    response = "Great! Please ask your next question and I'll be happy to help."
                    st.session_state.messages.append({"role": "assistant", "content": response})
                    st.session_state.awaiting_feedback = False
                    st.rerun()

                else:
                    st.session_state.messages.append({"role": "user", "content": prompt})
                    response = "Please respond with 'Yes' or 'No'. Would you like to ask anything else?"
                    st.session_state.messages.append({"role": "assistant", "content": response})

            elif st.session_state.awaiting_rating:
                try:
                    rating = int(prompt)
                    if 1 <= rating <= 10:
                        success = analyze_and_improve(rating)
                        st.session_state.messages.append({"role": "user", "content": f"Rating: {rating}/10"})
                        st.session_state.awaiting_rating = False

                        if rating < 5:
                            response = random.choice(feedback_responses["rating_responses"]["low"])
                        elif rating < 8:
                            response = random.choice(feedback_responses["rating_responses"]["medium"])
                        else:
                            response = random.choice(feedback_responses["rating_responses"]["high"])

                        if not success:
                            response += "\n\n⚠️ Note: Your rating wasn't saved due to a technical issue."

                        st.session_state.messages.append({"role": "assistant", "content": response})

                        goodbye = random.choice(feedback_responses["goodbye"])
                        st.session_state.messages.append({"role": "assistant", "content": goodbye})
                        st.session_state.session_complete = True
                        st.rerun()
                    else:
                        response = "Please enter a number between 1 and 10"
                        st.session_state.messages.append({"role": "assistant", "content": response})
                except ValueError:
                    response = "Please enter a number between 1 and 10"
                    st.session_state.messages.append({"role": "assistant", "content": response})

            else:
                st.session_state.messages.append({"role": "user", "content": prompt})

                with st.chat_message("assistant"):
                    with st.spinner("🤔 Thinking..."):
                        response = generate_response(prompt, st.session_state.pdf_text)
                        st.markdown(response)
                        st.session_state.messages.append({"role": "assistant", "content": response})

                        if not st.session_state.awaiting_rating:
                            more_questions_prompt = """
                            <div style='background-color:#f0f2f6; padding:10px; border-radius:5px;'>
                            <b>Do you have any more questions? (Yes/No)</b>
                            </div>
                            """
                            st.markdown(more_questions_prompt, unsafe_allow_html=True)
                            st.session_state.messages.append({"role": "assistant", "content": more_questions_prompt})
                            st.session_state.awaiting_feedback = True





